# Benchmark 1

We study the Hamiltonian of the Heisenberg model with periodic boundary conditions.

In [1]:
using CondensedMatterSOS
import MultivariatePolynomials as MP
@spin σ[1:3]
MP.monomials(σ[1], 0:2, consecutive=true)
heisenberg_hamiltonian(σ, true)

σᶻ₂σᶻ₃ + σʸ₂σʸ₃ + σˣ₂σˣ₃ + σᶻ₁σᶻ₃ + σᶻ₁σᶻ₂ + σʸ₁σʸ₃ + σʸ₁σʸ₂ + σˣ₁σˣ₃ + σˣ₁σˣ₂

Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

In [2]:
import Clarabel
solver = Clarabel.Optimizer

Clarabel.MOIwrapper.Optimizer

We can compute a lower bound `-2√2` to the ground state energy as follow:

In [3]:
function hamiltonian_energy(N, maxdegree, solver; symmetry=true, consecutive=false, kws...)
    @spin σ[1:N]
    G = Lattice1Group(N)
    @assert iseven(maxdegree)
    H = heisenberg_hamiltonian(σ, true)
    cone = NonnegPolyInnerCone{MOI.HermitianPositiveSemidefiniteConeTriangle}()
    certificate = SumOfSquares.Certificate.FixedBasis(
        cone,
        MonomialBasis(MP.monomials(σ[1], 0:div(maxdegree, 2), consecutive=consecutive)),
    )
    if symmetry
        certificate = Symmetry.Ideal(
            Symmetry.Pattern(G, Action(σ)),
            certificate,
        )
    end
    energy(H, maxdegree, solver; certificate, kws...)
end
bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
    symmetry = false,
    sparsity = SumOfSquares.Sparsity.NoPattern(),
)
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 50
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 127
  cones (total) = 2
    : Zero        = 1,  numel = 16
    : PSDTriangle = 1,  numel = 105

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       g

-5.999999993853594

We can see that the moment matrix uses all monomials:

In [4]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σᶻ₂
 σʸ₂
 σˣ₂
 σᶻ₁
 σʸ₁
 σˣ₁

# Symmetry reduction

We can reduce the computation using symmetry reduction as follows.

In [5]:
using CondensedMatterSOS

bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
)
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 4
  constraints   = 13
  nnz(P)        = 0
  nnz(A)        = 16
  cones (total) = 4
    : Zero        = 1,  numel = 4
    : PSDTriangle = 3,  numel = (3,3,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       g

-5.999999997057931

The reduction is obtained by block diagonalizing with a change of polynomial
basis to the isotypical basis.

In [6]:
[M.basis.polynomials for M in ν.blocks]

7-element Vector{Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}}:
 [(-0.7071067811865472 + 0.0im)σᶻ₂ + (-0.7071067811865475 + 0.0im)σᶻ₁]
 [(-0.7071067811865472 + 0.0im)σʸ₂ + (-0.7071067811865475 + 0.0im)σʸ₁]
 [(-0.7071067811865472 + 0.0im)σˣ₂ + (-0.7071067811865475 + 0.0im)σˣ₁]
 [(1.0 - 0.0im)]
 [(-0.7071067811865472 + 0.0im)σᶻ₂ + (0.7071067811865475 + 0.0im)σᶻ₁]
 [(-0.7071067811865472 + 0.0im)σʸ₂ + (0.7071067811865475 + 0.0im)σʸ₁]
 [(-0.7071067811865472 + 0.0im)σˣ₂ + (0.7071067811865475 + 0.0im)σˣ₁]

Let's try this for 3 sites. First without symmetry.

In [7]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
    symmetry = false,
)
@show bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 101
  constraints   = 247
  nnz(P)        = 0
  nnz(A)        = 256
  cones (total) = 2
    : Zero        = 1,  numel = 37
    : PSDTriangle = 1,  numel = 210

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       

-4.499999995399682

Now with symmetry.

In [8]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
)
@show bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 5
  constraints   = 22
  nnz(P)        = 0
  nnz(A)        = 40
  cones (total) = 5
    : Zero        = 1,  numel = 10
    : PSDTriangle = 4,  numel = (3,3,3,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost     

-4.499999996767557

Let's look at the isotypical basis.

In [9]:
[M.basis.polynomials for M in ν.blocks]

10-element Vector{Vector{MultivariatePolynomials.VectorPolynomial{ComplexF64, MultivariatePolynomials.Term{ComplexF64, CondensedMatterSOS.SpinMonomial}}}}:
 [(-0.5773502691896257 + 0.0im)σᶻ₃ + (-0.5773502691896257 + 0.0im)σᶻ₂ + (-0.5773502691896257 + 0.0im)σᶻ₁]
 [(-0.5773502691896257 + 0.0im)σʸ₃ + (-0.5773502691896257 + 0.0im)σʸ₂ + (-0.5773502691896257 + 0.0im)σʸ₁]
 [(-0.5773502691896257 + 0.0im)σˣ₃ + (-0.5773502691896257 + 0.0im)σˣ₂ + (-0.5773502691896257 + 0.0im)σˣ₁]
 [(1.0 - 0.0im)]
 [(-0.577350269189626 + 0.0im)σᶻ₃ + (0.2886751345948131 - 0.49999999999999994im)σᶻ₂ + (0.28867513459481275 + 0.5000000000000001im)σᶻ₁]
 [(-0.577350269189626 + 0.0im)σʸ₃ + (0.2886751345948131 - 0.49999999999999994im)σʸ₂ + (0.28867513459481275 + 0.5000000000000001im)σʸ₁]
 [(-0.577350269189626 + 0.0im)σˣ₃ + (0.2886751345948131 - 0.49999999999999994im)σˣ₂ + (0.28867513459481275 + 0.5000000000000001im)σˣ₁]
 [(-0.577350269189626 + 0.0im)σᶻ₃ + (0.28867513459481275 + 0.5000000000000001im)σᶻ₂ + (0.288675134594813

Now let's define a function for our common use case.

In [10]:
function f(N, d=1; verbose = 1, kws...)
    @show N
    println("***")
    @show d
    bound, _, ν = @time hamiltonian_energy(N, 2d, solver; kws...)
    @show bound
    block_sizes = map(ν.blocks) do M
        return length(M.basis.polynomials)
    end
    @show block_sizes
    if verbose >= 2
        for M in ν.blocks
            display(round.(M.basis.polynomials, digits=6))
        end
    end
    println("E/N = ", bound / N)
    println("------------------------------------")
    return bound / N
end

f (generic function with 2 methods)

With `d = 1`, we find a lower bound of `-3`:

In [11]:
lb = f(6, 1, consecutive = true, symmetry = true)

N = 6
***
d = 1
-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 8
  constraints   = 67
  nnz(P)        = 0
  nnz(A)        = 292
  cones (total) = 8
    : Zero        = 1,  numel = 46
    : PSDTriangle = 7,  numel = (3,3,3,3,...,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    

-2.999999998927544

Now with `d = 2`, we find `-2`:

In [12]:
lb = f(6, 2, consecutive = true, symmetry = true)

N = 6
***
d = 2
-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 46
  constraints   = 896
  nnz(P)        = 0
  nnz(A)        = 9975
  cones (total) = 25
    : Zero        = 1,  numel = 775
    : PSDTriangle = 24,  numel = (3,10,10,3,...,3)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10



-1.9999999494839378

Now with `d = 3`, we find `-1.8685`:

In [13]:
lb = f(6, 3, consecutive = true, symmetry = true)

N = 6
***
d = 3
-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 402
  constraints   = 5774
  nnz(P)        = 0
  nnz(A)        = 269426
  cones (total) = 31
    : Zero        = 1,  numel = 4875
    : PSDTriangle = 30,  numel = (21,78,21,3,...,36)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter

-1.868516999707432

| id     | irep 1 | irep 2 | irep 3 | irep 4 |
|--------|--------|--------|--------|--------|
| degree | 2      | 3      | 1      | 3      |
| mult 2 | 1      | 3      | 2      | 1      |
| mult 3 | 3      | 6      | 4      | 3      |
| mult 4 | 6      | 10     | 7      | 6      |
| mult 5 | 10     | 15     | 11     | 10     |
| mult 6 | 15     | 21     | 16     | 15     |
| mult 7 | 21     | 28     | 22     | 21     |

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*